# This pipeline is a fork of DockM8

In [1]:
#Import required libraries and scripts
from DockM8.docking_functions import *
from DockM8.rescoring_functions import *
from DockM8.consensus_methods import *
# from DockM8.scripts.dogsitescorer import *
# from DockM8.scripts.get_pocket import *
from tqdm.autonotebook import tqdm

In [2]:
protein_name  = 'protein_protoss_noligand.pdb'
ligand_library = 'ecft_scores_new_cleaned.sdf'
ref_ligand = 'ref_ligand.sdf'
pocket = 'protein_protoss_noligand_pocket.pdb'
snapshot_IDs = ['p9', 'p11']

In [3]:
HERE = Path(_dh[-1])
DATA = (HERE / "data")

# Move input data (protein pdb, docking library and reference ligand) to data directory
# software = (HERE / "software")
protein_file = (DATA  / protein_name)
ligand_library = (DATA / ligand_library)
ref_file = (DATA / ref_ligand)


OUTPUT = DATA / "results"

### Move snapshots to another different folders

### Load Ground truth data with 2D compounds ['ID', '2D structure', 'Activity score']

# Data-preprocessing

### Protein is prepared by [Protoss](https://proteins.plus/)

### Ligand library preparation by Gypsum-DL for 3D conformers generation

In [4]:
from data_preparation import run_gypsumdl


prepared_library_path = OUTPUT / f"{ligand_library.stem}_prepared.sdf"
run_gypsumdl(ligand_library, prepared_library_path)

Molecules are already prepared


In [5]:
df_prepared = PandasTools.LoadSDF(str(prepared_library_path))
df_prepared.head(5)

,ID,ROMol
0,HIPS6128,<rdkit.Chem.rdchem.Mol object at 0x7f62006b2420>
1,HIPS449,<rdkit.Chem.rdchem.Mol object at 0x7f61620bc3c0>
2,HIPS6989,<rdkit.Chem.rdchem.Mol object at 0x7f61620bc430>
3,HIPS7002,<rdkit.Chem.rdchem.Mol object at 0x7f61620bc4a0>
4,HIPS7000,<rdkit.Chem.rdchem.Mol object at 0x7f62006b2260>


In [6]:
docking_programs = [
                'GNINA', 
                'SMINA',
                'local_diffdock', 
                'PLANTS', 
                'flexx',
                ]


consensus_methods = []
n_poses = 10
exhaustiveness = 8

# Docking

1. Local DiffDock
2. PLANTS (Implemented by DockM8)
3. SMINA (Implemented by DockM8)
4. GNINA (Implemented by DockM8)
5. FlexX

NOTE : Output of docking step should have at least two columns 
 1. **ID** : Name of the compound , name of docking tool and number of pose e.g. (compoundX_diffdock_01)
 2. **Molecule** : Poses of every docking tool

In [7]:
from docking import docking
docking(
        docking_programs,
        protein_file,
        prepared_library_path,
        ref_file,
        exhaustiveness,
        n_poses
        )

Extracting ligand coordinates supports either SDF files or PDB files...
Compounds are already docked with GNINA v 1.0
Compounds are already docked with SMINA
Binding pocket is already extracted


Local DiffDock is running ...:   0%|          | 0/212 [00:00<?, ?it/s]

Compound HIPS6128 is already docked with Local DiffDock
Compound HIPS449 is already docked with Local DiffDock
Compound HIPS6989 is already docked with Local DiffDock
Compound HIPS7002 is already docked with Local DiffDock
Compound HIPS7000 is already docked with Local DiffDock
Compound HIPS6994 is already docked with Local DiffDock
Compound HIPS6991 is already docked with Local DiffDock
Compound HIPS6998 is already docked with Local DiffDock
Compound HIPS7006 is already docked with Local DiffDock
Compound HIPS7004 is already docked with Local DiffDock
Compound HIPS7001 is already docked with Local DiffDock
Compound HIPS6992 is already docked with Local DiffDock
Compound HIPS6981 is already docked with Local DiffDock
Compound HIPS6999 is already docked with Local DiffDock
Compound HIPS7242 is already docked with Local DiffDock
Compound HIPS470 is already docked with Local DiffDock
Compound HIPS6984 is already docked with Local DiffDock
Compound HIPS6990 is already docked with Local Dif

In [8]:
# @ TODO : Rescore poses with all rescoring functions

# Rescoring

### Choose wanted scoring function from the next list
- By default all SFs are selected, comment unwanted one

In [9]:
rescoring = [
    'gnina_rescoring', 
    'ad4',  
    'linf9', 
    'rtmscore', 
    'vinardo', 
    'chemplp',
    'rfscorevs_v1',
    'rfscorevs_v2',
    'rfscorevs_v3', 
    'vina_hydrophobic', 
    'vina_intra_hydrophobic',
    'scorch',
    'hyde', 
    ]
docked_library_path = OUTPUT / f"allposes.sdf"


### Run Rescoring

In [16]:
from rescoring import rescoring_function
  
docked_library_path = OUTPUT / f"allposes.sdf"

rescoring_function(
    rescoring,
    protein_file,
    docked_library_path,
    ref_file,
)

protein is already converted to mol2


Now rescoring with gnina_rescoring ... ⌛⌛ 
gnina_rescoring is already excuted
gnina_rescoring is already read


Now rescoring with ad4 ... ⌛⌛ 
ad4 is already excuted
ad4 is already read


Now rescoring with linf9 ... ⌛⌛ 
linf9 is already excuted
linf9 is already read


Now rescoring with rtmscore ... ⌛⌛ 
rtmscore is already excuted
rtmscore is already read


Now rescoring with vinardo ... ⌛⌛ 
vinardo is already excuted
vinardo is already read


Now rescoring with chemplp ... ⌛⌛ 
chemplp is already excuted
chemplp is already read


Now rescoring with rfscorevs_v1 ... ⌛⌛ 
rfscorevs_v1 is already excuted
rfscorevs_v1 is already read


Now rescoring with rfscorevs_v2 ... ⌛⌛ 
rfscorevs_v2 is already excuted
rfscorevs_v2 is already read


Now rescoring with rfscorevs_v3 ... ⌛⌛ 
rfscorevs_v3 is already excuted
rfscorevs_v3 is already read


Now rescoring with vina_hydrophobic ... ⌛⌛ 
vina_hydrophobic is already excuted
vina_hydrophobic is already read




In [69]:
from sklearn.preprocessing import MinMaxScaler

#load rescoring and ground truth and merging them together
rescoring_df = pd.read_csv(OUTPUT / "all_rescoring_results.csv")
rescoring_df[['ID', 'docking_program', 'pose']] = rescoring_df['ID'].str.split('_', expand=True)
true_score = PandasTools.LoadSDF(str(ligand_library))[['ID', 'score']]
merged_df = pd.merge(true_score, rescoring_df, on='ID', how='outer')
merged_df = merged_df.apply(pd.to_numeric, errors='ignore')


#normalize scores of merged dataframe
scaler = MinMaxScaler()
selected_columns = [col for col in merged_df.columns if col not in ['ID', 'docking_program', 'pose']]
merged_df_norm = pd.DataFrame(scaler.fit_transform(merged_df[selected_columns]), columns=selected_columns)
merged_df_norm[['ID', 'docking_program', 'pose']] = merged_df[['ID', 'docking_program', 'pose']]

# find the IDs that have less than 50 
non_sense_poses = merged_df['ID'].value_counts()[merged_df['ID'].value_counts() < 50]
print(f'There are some poses of {len(non_sense_poses)} compounds that have less than 50 poses\n')
# merged_df_norm.dropna(inplace=True)
# calculate the mean square error between every column and score column

# calculate the mean square error between every column and score column
for column in selected_columns:
    if column != 'score':
        merged_df_norm[column] = (merged_df_norm[column] - merged_df_norm['score']) ** 2
merged_df_norm['mean_square_error'] = merged_df_norm[selected_columns].sum(axis=1)
display(merged_df_norm)

# find the best pose for each compound for each docking tool
best_poses = merged_df_norm.groupby(['ID', 'docking_program'])['mean_square_error'].min().reset_index()
best_poses = pd.merge(best_poses, merged_df_norm, on=['ID', 'docking_program', 'mean_square_error'], how='left')

# fill unscored poses with 0
best_poses.fillna(0, inplace=True)

There are some poses of 8 compounds that have less than 50 poses



,score,CNNscore,CNNaffinity,gnina_affinity,ad4,LinF9,RTMScore,Vinardo,CHEMPLP,rfscore_v1,rfscore_v2,rfscore_v3,vina_hydrophobic,vina_intra_hydrophobic,SCORCH,HYDE,ID,docking_program,pose,mean_square_error
0,0.0,1.794214e-05,0.279491,0.048395,0.070259,0.053435,0.000000e+00,0.044673,0.052459,0.736009,0.804320,0.611000,0.196511,0.055847,0.000003,1.285707e-07,HIPS6128,localdiffdock,1,2.952419
1,0.0,7.455135e-03,0.453442,0.057590,0.089749,0.060453,0.000000e+00,0.051796,0.034383,0.676965,0.752329,0.559608,0.179551,0.133471,0.000003,1.285707e-07,HIPS6128,localdiffdock,2,3.056795
2,0.0,2.505176e-10,0.143511,0.221697,0.315708,0.223111,5.961414e-09,0.209543,0.047733,0.662647,0.613055,0.558593,0.231384,0.045023,0.000003,1.285707e-07,HIPS6128,localdiffdock,3,3.272008
3,0.0,1.265346e-06,0.188802,0.150647,0.216264,0.152345,2.595711e-07,0.146936,0.075534,0.734238,0.731691,0.595151,0.232597,0.141805,0.000003,1.285707e-07,HIPS6128,localdiffdock,4,3.366016
4,0.0,3.669128e-07,0.188630,0.117947,0.216727,0.117716,5.851325e-07,0.124031,0.528532,0.783581,0.780339,0.645008,0.499735,0.483676,0.000003,1.285707e-07,HIPS6128,localdiffdock,5,4.485926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10544,1.0,NaN,NaN,NaN,NaN,0.905721,1.319373e-01,NaN,0.951204,0.187022,0.270415,0.299230,0.575453,0.901062,0.209725,9.999995e-01,HIPS6787,PLANTS,6,6.431769
10545,1.0,NaN,NaN,NaN,NaN,0.908784,2.525256e-01,NaN,0.848774,0.121862,0.224349,0.251812,0.565911,0.957109,0.210464,9.999994e-01,HIPS6787,PLANTS,7,6.341590
10546,1.0,NaN,NaN,NaN,NaN,0.909134,2.681855e-01,NaN,0.943339,0.170075,0.228213,0.311579,0.578504,0.957109,0.211203,9.999976e-01,HIPS6787,PLANTS,8,6.577338
10547,1.0,NaN,NaN,NaN,NaN,0.907471,2.156403e-01,NaN,0.779988,0.090216,0.232727,0.213045,0.549923,0.957109,0.212809,9.999998e-01,HIPS6787,PLANTS,9,6.158929


# Consensus ranking methods (Implemented by DockM8)
### You can also choose the ranking methods according to you preference

### Run Ranking methods